# STOP THE COUNT!

Let us count only the votes emitted before 18:00.

In [ ]:
import pandas as pd
import dask.dataframe as dd

In [ ]:
# Your cluster here.

In [ ]:
client.restart()

In [ ]:
def get_winner(count_per_region: pd.DataFrame) -> pd.Series:
    results = list()
    for region, df in count_per_region.groupby("region"):
        results.append(
            {"region": region,
             "winner": df.set_index("vote").result.idxmax()}
        )

    winner_per_region = pd.DataFrame(results)
    delegates_per_region = pd.read_csv("data/region_delegates.csv")
    winner_region_delegates = pd.merge(winner_per_region, delegates_per_region, on="region")
    return (winner_region_delegates
            .groupby("winner")
            .delegates.sum()
            .sort_values(ascending=False))    

In [ ]:
data = dd.read_csv("data/votes_*.csv",
                   usecols=["timestamp", "region", "vote"],
                   parse_dates=["timestamp"],
                   dtype={"region": "category", "vote": "category"})
data["result"] = 1

## Is there a basis for the claim?

In [ ]:
data["hour"] = data["timestamp"].dt.hour

In [ ]:
(data[data["vote"] == "blue"]
 .groupby("hour")
 .result.agg("count")
 .compute()
 .plot(kind="bar"))

The one thing that changes is that here we will filter before aggregating.

In [ ]:
count_per_region_before_17 = (
    data[data.timestamp.dt.hour < 17]
    .groupby(["region", "vote"])
    .result.agg("count")
    .reset_index()
)

In [ ]:
count_per_region_before_17_computed = count_per_region_before_17.compute()

In [ ]:
get_winner(count_per_region_before_17_computed)